In [21]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/IITP/sohyun/creditcard_prediction/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/IITP/sohyun/creditcard_prediction/data'
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data


In [22]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [23]:
import torch
import os
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image
import h5py
import numpy as np
import collections
import numbers
import math
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import os
import argparse
from torch.backends import cudnn
from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score
import time
import datetime
from torch.autograd import grad
from torch.autograd import Variable
import matplotlib.pyplot as plt
import IPython
from tqdm import tqdm

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [24]:
train_df = pd.read_csv('./train.csv')
train_df = train_df.drop(columns=['ID'])
val_df = pd.read_csv('./val.csv')
val_df = val_df.drop(columns=['ID'])
test_df = pd.read_csv('./test.csv')
test_df = test_df.drop(columns=['ID'])

#-------------------#
#---# Normalize #---#
#-------------------#
# case 1 - standardscaler
scaler_n = StandardScaler()
scaler_n.fit(train_df)

val_x = val_df.drop(columns=['Class'])
train_x_scaleN = pd.DataFrame(scaler_n.transform(train_df), columns = train_df.columns) # 확인 : train_x_scaleN.mean(), train_x_scaleN.var()
val_x_scaleN = pd.DataFrame(scaler_n.transform(val_x), columns = val_x.columns)
test_x_scaleN = pd.DataFrame(scaler_n.transform(test_df), columns = test_df.columns)

train_df = train_x_scaleN
val_df = pd.concat([val_x_scaleN, pd.DataFrame(val_df['Class'])], axis=1)
test_df = test_x_scaleN

In [25]:
def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x, volatile=volatile)

def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [26]:
class KDD99Loader(object):
    def __init__(self, data_path, mode="train"):
        self.mode=mode

        self.train = train_df.values
        self.train_labels = np.tile(0, len(train_df))

        self.test = val_df.drop(columns = ['Class']).values
        self.test_labels = val_df['Class'].values

    def __len__(self):
        """
        Number of images in the object dataset.
        """
        if self.mode == "train":
            return self.train.shape[0]
        else:
            return self.test.shape[0]

    def __getitem__(self, index):
        if self.mode == "train":
            return np.float32(self.train[index]), np.float32(self.train_labels[index])
        else:
           return np.float32(self.test[index]), np.float32(self.test_labels[index])
        

def get_loader(data_path, batch_size, mode='train'):
    """Build and return data loader."""

    dataset = KDD99Loader(data_path, mode)

    shuffle = False
    if mode == 'train':
        shuffle = True

    data_loader = DataLoader(dataset=dataset,
                             batch_size=batch_size,
                             shuffle=shuffle)
    return data_loader

In [27]:
class Cholesky(torch.autograd.Function):
    def forward(ctx, a):
        l = torch.cholesky(a, False)
        ctx.save_for_backward(l)
        return l
    def backward(ctx, grad_output):
        l, = ctx.saved_variables
        linv = l.inverse()
        inner = torch.tril(torch.mm(l.t(), grad_output)) * torch.tril(
            1.0 - Variable(l.data.new(l.size(1)).fill_(0.5).diag()))
        s = torch.mm(linv.t(), torch.mm(inner, linv))
        return s
    
class DaGMM(nn.Module):
    """Residual Block."""
    def __init__(self, n_gmm = 4, latent_dim=12):
        super(DaGMM, self).__init__()

        layers = []
        # layers += [nn.Linear(118,60)]
        # layers += [nn.Tanh()]        
        # layers += [nn.Linear(60,30)]
        # layers += [nn.Tanh()]        
        layers += [nn.Linear(30,25)]
        layers += [nn.Tanh()]         
        layers += [nn.Linear(25,20)]
        layers += [nn.Tanh()]         
        layers += [nn.Linear(20,10)]
        # layers += [nn.Tanh()]         
        # layers += [nn.Linear(15,10)]

        self.encoder = nn.Sequential(*layers)

        layers = []
        # layers += [nn.Linear(10,15)]
        # layers += [nn.Tanh()]        
        layers += [nn.Linear(10,20)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(20,25)]
        layers += [nn.Tanh()]        
        layers += [nn.Linear(25,30)]
        # layers += [nn.Tanh()]        
        # layers += [nn.Linear(60,118)]

        self.decoder = nn.Sequential(*layers)

        layers = []
        layers += [nn.Linear(latent_dim,10)]
        layers += [nn.Tanh()]        
        layers += [nn.Dropout(p=0.5)]        
        layers += [nn.Linear(10,n_gmm)]
        layers += [nn.Softmax(dim=1)]


        self.estimation = nn.Sequential(*layers)

        self.register_buffer("phi", torch.zeros(n_gmm))
        self.register_buffer("mu", torch.zeros(n_gmm,latent_dim))
        self.register_buffer("cov", torch.zeros(n_gmm,latent_dim,latent_dim))

    def relative_euclidean_distance(self, a, b):
        return (a-b).norm(2, dim=1) / a.norm(2, dim=1)

    def forward(self, x):

        enc = self.encoder(x)

        dec = self.decoder(enc)

        rec_cosine = F.cosine_similarity(x, dec, dim=1)
        rec_euclidean = self.relative_euclidean_distance(x, dec)

        z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)

        gamma = self.estimation(z)

        return enc, dec, z, gamma

    def compute_gmm_params(self, z, gamma):
        N = gamma.size(0)
        # K
        sum_gamma = torch.sum(gamma, dim=0)

        # K
        phi = (sum_gamma / N)

        self.phi = phi.data

 
        # K x D
        mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1)
        self.mu = mu.data
        # z = N x D
        # mu = K x D
        # gamma N x K

        # z_mu = N x K x D
        z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

        # z_mu_outer = N x K x D x D
        z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2)

        # K x D x D
        cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim = 0) / sum_gamma.unsqueeze(-1).unsqueeze(-1)
        self.cov = cov.data

        return phi, mu, cov
        
    def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
        if phi is None:
            phi = to_var(self.phi)
        if mu is None:
            mu = to_var(self.mu)
        if cov is None:
            cov = to_var(self.cov)

        k, D, _ = cov.size()

        z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

        cov_inverse = []
        det_cov = []
        cov_diag = 0
        eps = 1e-12
        for i in range(k):
            # K x D x D
            cov_k = cov[i] + to_var(torch.eye(D)*eps)
            cov_inverse.append(torch.inverse(cov_k).unsqueeze(0))

            #det_cov.append(np.linalg.det(cov_k.data.cpu().numpy()* (2*np.pi)))
            det_cov.append((Cholesky.apply(cov_k.cpu() * (2*np.pi)).diag().prod()).unsqueeze(0))
            cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

        # K x D x D
        cov_inverse = torch.cat(cov_inverse, dim=0)
        # K
        det_cov = torch.cat(det_cov).cuda()
        #det_cov = to_var(torch.from_numpy(np.float32(np.array(det_cov))))

        # N x K
        exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1) * cov_inverse.unsqueeze(0), dim=-2) * z_mu, dim=-1)
        # for stability (logsumexp)
        max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0]

        exp_term = torch.exp(exp_term_tmp - max_val)

        # sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (det_cov).unsqueeze(0), dim = 1) + eps)
        sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt(det_cov)).unsqueeze(0), dim = 1) + eps)
        # sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt((2*np.pi)**D * det_cov)).unsqueeze(0), dim = 1) + eps)


        if size_average:
            sample_energy = torch.mean(sample_energy)

        return sample_energy, cov_diag


    def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):

        recon_error = torch.mean((x - x_hat) ** 2)

        phi, mu, cov = self.compute_gmm_params(z, gamma)

        sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)

        loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag

        return loss, sample_energy, recon_error, cov_diag

In [28]:
class Solver(object):
    DEFAULTS = {}   
    def __init__(self, data_loader, config):
        # Data loader
        self.__dict__.update(Solver.DEFAULTS, **config)
        self.data_loader = data_loader

        # Build tensorboard if use
        self.build_model()
        # if self.use_tensorboard:
        #     self.build_tensorboard()

        # Start with trained model
        if self.pretrained_model:
            self.load_pretrained_model()

    def build_model(self):
        # Define model
        self.dagmm = DaGMM(self.gmm_k)

        # Optimizers
        self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr)

        # Print networks
        self.print_network(self.dagmm, 'DaGMM')

        if torch.cuda.is_available():
            self.dagmm.cuda()

    def print_network(self, model, name):
        num_params = 0
        for p in model.parameters():
            num_params += p.numel()
        print(name)
        print(model)
        print("The number of parameters: {}".format(num_params))

    def load_pretrained_model(self):
        self.dagmm.load_state_dict(torch.load(os.path.join(
            self.model_save_path, '{}_dagmm.pth'.format(self.pretrained_model))))

        # print("phi", self.dagmm.phi,"mu",self.dagmm.mu, "cov",self.dagmm.cov)

        print('loaded trained models (step: {})..!'.format(self.pretrained_model))

    # def build_tensorboard(self):
    #     from logger import Logger
    #     self.logger = Logger(self.log_path)

    def reset_grad(self):
        self.dagmm.zero_grad()

    def to_var(self, x, volatile=False):
        if torch.cuda.is_available():
            x = x.cuda()
        return Variable(x, volatile=volatile)

    def train(self):
        iters_per_epoch = len(self.data_loader)

        # Start with trained model if exists
        if self.pretrained_model:
            start = int(self.pretrained_model.split('_')[0])
        else:
            start = 0

        # Start training
        iter_ctr = 0
        start_time = time.time()
      
        self.ap_global_train = np.array([0,0,0])
        for e in range(start, self.num_epochs):
            sum_total_loss = 0; sum_sample_energy = 0; sum_recon_error = 0; sum_cov_diag = 0 
            for i, (input_data, labels) in enumerate(tqdm(self.data_loader)):
                iter_ctr += 1
                start = time.time()

                input_data = self.to_var(input_data)

                total_loss,sample_energy, recon_error, cov_diag = self.dagmm_step(input_data)
                # Logging
                loss = {}
                loss['total_loss'] = total_loss.data.item()
                loss['sample_energy'] = sample_energy.item()
                loss['recon_error'] = recon_error.item()
                loss['cov_diag'] = cov_diag.item()

                sum_total_loss += total_loss.data.item()
                sum_sample_energy += sample_energy.item()
                sum_recon_error += recon_error.item()
                sum_cov_diag += cov_diag.item()


                # Print out log info
                # if (i+1) % self.log_step == 0:
                #     elapsed = time.time() - start_time
                #     total_time = ((self.num_epochs*iters_per_epoch)-(e*iters_per_epoch+i)) * elapsed/(e*iters_per_epoch+i+1)
                #     epoch_time = (iters_per_epoch-i)* elapsed/(e*iters_per_epoch+i+1)
                    
                #     epoch_time = str(datetime.timedelta(seconds=epoch_time))
                #     total_time = str(datetime.timedelta(seconds=total_time))
                #     elapsed = str(datetime.timedelta(seconds=elapsed))

                #     lr_tmp = []
                #     for param_group in self.optimizer.param_groups:
                #         lr_tmp.append(param_group['lr'])
                #     tmplr = np.squeeze(np.array(lr_tmp))

                #     log = "Elapsed {}/{} -- {} , Epoch [{}/{}], Iter [{}/{}], lr {}".format(
                #         elapsed,epoch_time,total_time, e+1, self.num_epochs, i+1, iters_per_epoch, tmplr)

                #     for tag, value in loss.items():
                #         log += ", {}: {:.4f}".format(tag, value)

                #     IPython.display.clear_output()
                #     print(log)

                    # if self.use_tensorboard:
                    #     for tag, value in loss.items():
                    #         self.logger.scalar_summary(tag, value, e * iters_per_epoch + i + 1)
                    # else:
                    #     plt_ctr = 1
                    #     if not hasattr(self,"loss_logs"):
                    #         self.loss_logs = {}
                    #         for loss_key in loss:
                    #             self.loss_logs[loss_key] = [loss[loss_key]]
                    #             plt.subplot(2,2,plt_ctr)
                    #             plt.plot(np.array(self.loss_logs[loss_key]), label=loss_key)
                    #             plt.legend()
                    #             plt_ctr += 1
                    #     else:
                    #         for loss_key in loss:
                    #             self.loss_logs[loss_key].append(loss[loss_key])
                    #             plt.subplot(2,2,plt_ctr)
                    #             plt.plot(np.array(self.loss_logs[loss_key]), label=loss_key)
                    #             plt.legend()
                    #             plt_ctr += 1

                    #     plt.show()

                # print("phi", self.dagmm.phi,"mu",self.dagmm.mu, "cov",self.dagmm.cov)
                # Save model checkpoints
            
                if (i+1) % self.model_save_step == 0:
                    torch.save(self.dagmm.state_dict(),
                        os.path.join(self.model_save_path, '{}_{}_dagmm.pth'.format(e+1, i+1)))
                    
            print("total_e", sum_total_loss)
            print("", sum_sample_energy)
            print("re", sum_recon_error)
            print("cov", sum_cov_diag)
            print("--")
            wandb.log({
                "total loss": sum_total_loss,
                "sample energy": sum_sample_energy,
                "recon error": sum_recon_error,
                "cov": sum_cov_diag
            })

    def dagmm_step(self, input_data):
        self.dagmm.train()
        enc, dec, z, gamma = self.dagmm(input_data)

        total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma, self.lambda_energy, self.lambda_cov_diag)

        self.reset_grad()
        total_loss.backward()

        torch.nn.utils.clip_grad_norm_(self.dagmm.parameters(), 5)
        self.optimizer.step()

        return total_loss,sample_energy, recon_error, cov_diag

    def test(self):
        print("======================TEST MODE======================")
        self.dagmm.eval()
        self.data_loader.dataset.mode="train"

        N = 0
        mu_sum = 0
        cov_sum = 0
        gamma_sum = 0

        for it, (input_data, labels) in enumerate(self.data_loader):
            input_data = self.to_var(input_data)
            enc, dec, z, gamma = self.dagmm(input_data)
            phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)
            
            batch_gamma_sum = torch.sum(gamma, dim=0)
            
            gamma_sum += batch_gamma_sum
            mu_sum += mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
            cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
            
            N += input_data.size(0)
            
        train_phi = gamma_sum / N
        train_mu = mu_sum / gamma_sum.unsqueeze(-1)
        train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)

        # print("N:",N)
        # print("phi :\n",train_phi)
        # print("mu :\n",train_mu)
        # print("cov :\n",train_cov)

        train_energy = []
        train_labels = []
        train_z = []
        for it, (input_data, labels) in enumerate(self.data_loader):
            input_data = self.to_var(input_data)
            enc, dec, z, gamma = self.dagmm(input_data)
            sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
            
            train_energy.append(sample_energy.data.cpu().numpy())
            train_z.append(z.data.cpu().numpy())
            train_labels.append(labels.numpy())


        train_energy = np.concatenate(train_energy,axis=0)
        train_z = np.concatenate(train_z,axis=0)
        train_labels = np.concatenate(train_labels,axis=0)


        self.data_loader.dataset.mode="test"
        test_energy = []
        test_labels = []
        test_z = []
        for it, (input_data, labels) in enumerate(self.data_loader):
            input_data = self.to_var(input_data)
            enc, dec, z, gamma = self.dagmm(input_data)
            sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
            test_energy.append(sample_energy.data.cpu().numpy())
            test_z.append(z.data.cpu().numpy())
            test_labels.append(labels.numpy())

        test_energy = np.concatenate(test_energy,axis=0)
        test_z = np.concatenate(test_z,axis=0)
        test_labels = np.concatenate(test_labels,axis=0)

        combined_energy = np.concatenate([train_energy, test_energy], axis=0)
        combined_labels = np.concatenate([train_labels, test_labels], axis=0)

        thresh = np.percentile(combined_energy, 100 - 0.1)
        print("Threshold :", thresh)

        pred = (test_energy > thresh).astype(int)
        gt = test_labels.astype(int)
        print("================================sum", sum(gt))
        from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score

        accuracy = accuracy_score(gt,pred)
        precision, recall, f_score, support = prf(gt, pred, average='macro')
        
        print("Accuracy : {:0.4f}, Precision : {:0.4f}, Recall : {:0.4f}, F-score : {:0.4f}".format(accuracy, precision, recall, f_score))
        
        return accuracy, precision, recall, f_score

In [29]:
import os
import argparse
from torch.backends import cudnn

def str2bool(v):
    return v.lower() in ('true')

def main(config):
    # For fast training
    cudnn.benchmark = True

    # Create directories if not exist
    mkdir(config.log_path)
    mkdir(config.model_save_path)

    data_loader = get_loader(config.data_path, batch_size=config.batch_size, mode=config.mode)
    
    # Solver
    solver = Solver(data_loader, vars(config))

    if config.mode == 'train':
        solver.train()
    elif config.mode == 'test':
        solver.test()

    return solver
    
if __name__ == '__main__':
    wandb.init()
    import easydict
    args = easydict.EasyDict({
        "num_epochs" : 40,
        "batch_size" : 4096,
        "gmm_k" : 100, #3
        "lambda_energy" : 0.1,
        "lambda_cov_diag" : 0.005,
        # "pretrained_model" : '',
        "pretrained_model" : None,
        "mode" : 'train',
        # "mode" : "test",
        # "data_path" : "./kdd_cup.npz",
        "data_path" : None, 
        "use_tensorboard" : False,
        "log_path" : './logs',
        "model_save_path" : './models',
        "log_step" : 194//4,
        "sample_step" : 194,
        "model_save_step" : 194,
        "lr" : 1e-7, #2e-5
        "wd" : None
    })
    config = args
 
    print('------------ Options -------------')
    for k, v in sorted(args.items()):
        print('%s: %s' % (str(k), str(v)))
    print('-------------- End ----------------')

    solver = main(config)
    solver.test()

------------ Options -------------
batch_size: 4096
data_path: None
gmm_k: 100
lambda_cov_diag: 0.005
lambda_energy: 0.1
log_path: ./logs
log_step: 48
lr: 1e-07
mode: train
model_save_path: ./models
model_save_step: 194
num_epochs: 40
pretrained_model: None
sample_step: 194
use_tensorboard: False
wd: None
-------------- End ----------------
DaGMM
DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=30, out_features=25, bias=True)
    (1): Tanh()
    (2): Linear(in_features=25, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=10, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=25, bias=True)
    (3): Tanh()
    (4): Linear(in_features=25, out_features=30, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_

  0%|          | 0/28 [00:00<?, ?it/s]<ipython-input-27-16826e21c896>:3: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1615.)
  l = torch.cholesky(a, False)
<ipython-input-27-16826e21c896>:7: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  l, = ctx.saved_variables
100%|██████████| 28/28 [00:12<00:00,  2.24it/s]


total_e 18871.164428710938
 -109.31263065338135
re 28.706895887851715
cov 3770677.796875
--


100%|██████████| 28/28 [00:04<00:00,  5.78it/s]


total_e 18872.585327148438
 -109.30487179756165
re 28.69761770963669
cov 3770963.71875
--


...

total_e 18711.672973632812
 -108.88088345527649
re 28.70246559381485
cov 3738771.7421875
--


100%|██████████| 28/28 [00:04<00:00,  6.01it/s]


total_e 18707.491943359375
 -108.85461258888245
re 28.706371545791626
cov 3737934.328125
--


100%|██████████| 28/28 [00:04<00:00,  5.87it/s]


total_e 18698.524963378906
 -108.8389663696289
re 28.724519729614258
cov 3736136.9296875
--
======================TEST MODE======================
Threshold : 27.631006240844727
================================sum 30
Accuracy : 0.9991, Precision : 0.7855, Recall : 0.7665, F-score : 0.7756
